In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import f1_score

# Define paths (update these to your actual paths)
base_dir = "/home/shravan/documents/deeplearning/github/ComputerVision-Research/finetuning/roam2/data"
support_dir = os.path.join(base_dir,  "support_samples/")
eval_dir = os.path.join(base_dir, "evaluation_data")
support_csv = os.path.join(base_dir, "support.csv")
eval_csv = os.path.join(base_dir, "solutionAugust5.csv")

# Custom Dataset
class VehicleDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['id']
        img_path = os.path.join(self.img_dir, f"{img_name}.jpg")
        image = Image.open(img_path).convert('RGB')
        label = self.data.iloc[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Data Augmentation and Normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
support_dataset = VehicleDataset(support_csv, support_dir, transform)
eval_dataset = VehicleDataset(eval_csv, eval_dir, transform)

support_loader = DataLoader(support_dataset, batch_size=6, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# # Model
# model = models.resnet50(pretrained=True)
# model.fc = nn.Linear(model.fc.in_features, 3)  # 3-way classification
# model = model.to(device)

In [3]:
# Load the pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the last few
for name, param in model.named_parameters():
    if "layer4" not in name and "fc" not in name:  # Adjust layer4 and fc as required
        param.requires_grad = False

# Modify the fully connected layer for your classification task
model.fc = nn.Linear(model.fc.in_features, 3)  # 3-way classification

# Move the model to the appropriate device
model = model.to(device)

# Verify which layers are trainable
# for name, param in model.named_parameters():
#     print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

/home/shravan/anaconda3/envs/hf_dev/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shravan/anaconda3/envs/hf_dev/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, dataloader, criterion, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Only compute loss for classes 1 and 2
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Geometric Mean Weighted F1-Score
def gm_weighted_f1_score(y_true, y_pred):
    f1_scores = f1_score(y_true, y_pred, average=None)
    return np.prod(f1_scores) ** (1/len(f1_scores))

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    score = gm_weighted_f1_score(all_labels, all_preds)
    print(f"Geometric Mean Weighted F1-Score: {score:.4f}")
    return all_preds, all_labels

In [5]:
%%time
# Train the model
train_model(model, support_loader, criterion, optimizer, num_epochs=100)

Epoch 1/100, Loss: 0.6305
Epoch 2/100, Loss: 0.0061
Epoch 3/100, Loss: 0.0005
Epoch 4/100, Loss: 0.0048
Epoch 5/100, Loss: 0.0000
Epoch 6/100, Loss: 0.0039
Epoch 7/100, Loss: 0.0001
Epoch 8/100, Loss: 0.0013
Epoch 9/100, Loss: 0.0001
Epoch 10/100, Loss: 0.0003
Epoch 11/100, Loss: 0.8094
Epoch 12/100, Loss: 0.0040
Epoch 13/100, Loss: 0.0129
Epoch 14/100, Loss: 0.0312
Epoch 15/100, Loss: 0.1099
Epoch 16/100, Loss: 0.0060
Epoch 17/100, Loss: 0.0800
Epoch 18/100, Loss: 0.0103
Epoch 19/100, Loss: 0.0290
Epoch 20/100, Loss: 0.1714
Epoch 21/100, Loss: 0.0814
Epoch 22/100, Loss: 0.0041
Epoch 23/100, Loss: 0.0054
Epoch 24/100, Loss: 0.0191
Epoch 25/100, Loss: 0.0063
Epoch 26/100, Loss: 0.0024
Epoch 27/100, Loss: 0.0009
Epoch 28/100, Loss: 0.0020
Epoch 29/100, Loss: 0.0010
Epoch 30/100, Loss: 0.0002
Epoch 31/100, Loss: 0.0031
Epoch 32/100, Loss: 0.0003
Epoch 33/100, Loss: 0.0003
Epoch 34/100, Loss: 0.0003
Epoch 35/100, Loss: 0.0001
Epoch 36/100, Loss: 0.0011
Epoch 37/100, Loss: 0.0002
Epoch 38/1

In [19]:
# Evaluate the model
preds, labels = evaluate_model(model, eval_loader)

# Create submission
submission = pd.DataFrame({
    'id': eval_dataset.data['id'],
    'label': preds
})
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")

Geometric Mean Weighted F1-Score: 0.0000
Submission file created: submission.csv


In [20]:
submission.groupby('label').count()

,id
label,
0,4957
2,43


In [8]:
def infer_from_dataset(model, dataset, index):
    # Get the image and label from the dataset
    image, label = dataset[index]
    
    # Add batch dimension (1, C, H, W)
    image = image.unsqueeze(0)
    
    # Move image to the same device as the model
    image = image.to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # Forward pass
        output = model(image)
        print(output)
        # Get the predicted class (index with the highest score)
        _, predicted_class = torch.max(output, 1)
    
    return predicted_class.item() + 1, label  # Add 1 to match your label indexing

In [9]:
# Example usage
index = 4510  # Choose the index of the image you want to infer
predicted_label, true_label = infer_from_dataset(model, eval_dataset, index)
print(f"Predicted Label: {predicted_label}")
print(f"True Label: {true_label}")

tensor([[-3.2292,  0.8020,  3.0162]], device='cuda:0')
Predicted Label: 3
True Label: 0


In [14]:
def infer_from_dataloader(model, dataloader, batch_index=0):
    # Get the batch
    for i, (images, labels) in enumerate(dataloader):
        if i == batch_index:
            break
    
    # Move images to the same device as the model
    images = images.to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    with torch.no_grad():
        # Forward pass
        outputs = model(images)
        
        # Get the predicted classes (indices with the highest scores)
        _, predicted_classes = torch.min(outputs, 1)
    
    # Add 1 to match your label indexing
    predicted_classes = predicted_classes.cpu().numpy()
    
    score = gm_weighted_f1_score(labels.numpy(), predicted_classes)
    print(f"Geometric Mean Weighted F1-Score: {score:.4f}")
    
    return predicted_classes, labels.numpy()

In [17]:
# Example usage
predicted_labels, true_labels = infer_from_dataloader(model, eval_loader, batch_index=32)
print(f"Predicted Labels: {predicted_labels}")
print(f"True Labels: {true_labels}")

Geometric Mean Weighted F1-Score: 1.0000
Predicted Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
True Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
